In [1]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip, VideoClip
import tempfile
import os

In [2]:
def process_frame(frame, object_img, sift, flann):
    # Convert frames to grayscale
    object_gray = cv2.cvtColor(object_img, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Find keypoints and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(object_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(frame_gray, None)
    
    if descriptors1 is None or descriptors2 is None:
        return frame
    
    # Match descriptors
    try:
        matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    except:
        return frame
    
    # Apply Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    # Draw border if enough matches are found
    if len(good_matches) >= 4:
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Find homography
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        if H is not None:
            # Get dimensions of object image
            h, w = object_gray.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            
            # Transform corners to frame coordinates
            dst = cv2.perspectiveTransform(pts, H)
            
            # Draw border
            frame = cv2.polylines(frame, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
    
    # Draw matches
    match_img = cv2.drawMatches(object_img, keypoints1, frame, keypoints2, good_matches, None,
                               flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return match_img

def process_video(video_path, object_path, output_path):
    # Read the object image
    object_img = cv2.imread(object_path)
    if object_img is None:
        raise ValueError("Could not read object image")
    
    # Initialize SIFT and FLANN
    sift = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # Read the input video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Could not open video file")
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Calculate new dimensions for the output
    new_width = width * 2  # Double width to accommodate matches
    new_height = height
    
    # List of codecs to try
    codecs = [
        ('XVID', '.avi'),
        ('MJPG', '.avi'),
        ('MP4V', '.mp4'),
        ('DIVX', '.avi'),
        ('X264', '.mp4')
    ]
    
    # Try different codecs until one works
    out = None
    temp_output = None
    
    for codec, ext in codecs:
        try:
            temp_output = f'temp_output{ext}'
            fourcc = cv2.VideoWriter_fourcc(*codec)
            out = cv2.VideoWriter(temp_output, fourcc, fps, (new_width, new_height))
            
            # Test if VideoWriter was successfully created
            if out is not None and out.isOpened():
                print(f"Successfully created video writer with codec: {codec}")
                break
            else:
                out = None
                if os.path.exists(temp_output):
                    os.remove(temp_output)
        except Exception as e:
            print(f"Failed to create video writer with codec {codec}: {str(e)}")
            if out is not None:
                out.release()
            if os.path.exists(temp_output):
                os.remove(temp_output)
    
    if out is None:
        raise ValueError("Could not create video writer with any codec")
    
    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print("Starting video processing...")
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Process frame
            processed_frame = process_frame(frame, object_img, sift, flann)
            
            # Ensure the processed frame has the correct dimensions
            if processed_frame.shape[1] != new_width or processed_frame.shape[0] != new_height:
                processed_frame = cv2.resize(processed_frame, (new_width, new_height))
            
            # Ensure the frame is in the correct color format (BGR)
            if len(processed_frame.shape) == 2:
                processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)
            
            # Write the frame
            out.write(processed_frame)
            
            # Update progress
            frame_count += 1
            if frame_count % 10 == 0:
                print(f"Processing frame {frame_count}/{total_frames} ({(frame_count/total_frames)*100:.1f}%)", end='\r')
            
            # Optional: Display the frame while processing
            display_frame = cv2.resize(processed_frame, (new_width//2, new_height//2))
            cv2.imshow('Processing...', display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
    except Exception as e:
        print(f"\nError during video processing: {str(e)}")
        raise
    
    finally:
        # Release everything
        cap.release()
        out.release()
        cv2.destroyAllWindows()
    
    print("\nVideo processing completed. Checking output...")
    
    # Verify the temporary file was created and has content
    if not os.path.exists(temp_output) or os.path.getsize(temp_output) == 0:
        raise ValueError("Failed to create valid output video file")
    
    print("Adding audio...")
    
    try:
        # Load the original video with audio
        original = VideoFileClip(video_path)
        # Load the processed video
        processed = VideoFileClip(temp_output)
        
        # Set the audio of the processed video to the original audio
        final_video = processed.set_audio(original.audio)
        
        # Write the final video
        final_video.write_videofile(output_path, 
                                  codec='libx264', 
                                  audio_codec='aac',
                                  temp_audiofile="temp-audio.m4a",
                                  remove_temp=True)
        
        # Close the clips
        original.close()
        processed.close()
        final_video.close()
        
        # Remove temporary video file
        if os.path.exists(temp_output):
            os.remove(temp_output)
            
        print("Video processing completed successfully!")
        
    except Exception as e:
        print(f"Error while reattaching audio: {e}")
        # If audio reattachment fails, just copy the video without audio
        if os.path.exists(temp_output):
            import shutil
            shutil.copy2(temp_output, output_path)
            os.remove(temp_output)
            print("Saved video without audio due to audio processing error")

In [3]:
video_path = 'video-o.mp4'    # Path to your input video
object_path = 'VObject.jpg'        # Path to your object image
output_path = 'output_video.mp4'  # Path for the output video

try:
    process_video(video_path, object_path, output_path)
except Exception as e:
    print(f"An error occurred: {e}")

Successfully created video writer with codec: MJPG
Starting video processing...
Processing frame 60/66 (90.9%)
Video processing completed. Checking output...
Adding audio...
Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Video processing completed successfully!
